In [15]:
import asyncio
from aioreactive.core import start, FuncSink
from aioreactive.producer import Producer
import aioreactive.producer.operators as ops

In [22]:
# define an abbreviation for asking the event loop to run our coroutine
def my_run(f):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(f())
    
# define an async version of the print function
async def aprint(value): # why does this need to be async?
    print(value)

# a simple source to listen to    
source = Producer.from_iterable(range(10))

To run, your code must be wrapped in a coroutine (function defined with `async def`) and then fed to the event loop to execute. 

In [25]:
async def f():
    stream = await start(source, FuncSink(aprint))
    
my_run(f)

0
1
2
3
4
5
6
7
8
9


In [26]:
async def f():
    async with start(source) as stream:
        async for x in stream:
            print(x)
    
my_run(f)

0
1
2
3
4
5
6
7
8
9


In [28]:
# my own attempt to just get the next value - fails; 
# sent an email to aioreactive author Dag Brattli asking how to do it properly
async def f():
    async with start(source) as stream:
        while True:
            x = await stream.__anext__()
            print(x)
    
my_run(f)

AttributeError: 'AsyncSingleStream' object has no attribute '__anext__'

In [29]:
async def main():
    xs = Producer.from_iterable(range(10))

    # Split into odds and evens
    odds = xs | ops.filter(lambda x: x % 2 == 1)
    evens = xs | ops.filter(lambda x: x % 2 == 0)

    async def mysink(value): # why does this need to be async?
        print(value)

    await start(odds, FuncSink(mysink))
    await start(evens, FuncSink(mysink))

my_run(main)

1
3
5
7
9
0
2
4
6
8
